In [2]:
import numpy as np
from numpy.linalg import inv

# Linear Dynamics
$$\dot{x} = A\vec{x} + B\vec{u} \\
 \vec{y} = C\vec{x}$$

$\vec{x}$ is the state variables. $v_{x}, v_{y}$ is world linear velocity of the robot in m/s, $\omega$ is the angular velocity of the robot around the z-axis in rad/s.
$$
\vec{x} = \begin{bmatrix}
v_{x} \\ 
v_{y} \\
\omega_{z}
\end{bmatrix}
$$

$\vec{u}$ is the input vector. $\omega_{iFL}, \omega_{iFR}, \omega_{iBL}, \omega_{iBR}$ is commanded angular velocity of the front left, front right, back left, back right wheels of the robot (will refer to those positions for the remainder).
$$
\vec{u} = \begin{bmatrix}
\omega_{iFL} \\
\omega_{iFR} \\
\omega_{iBL} \\
\omega_{iBR}
\end{bmatrix}
$$

$\vec{y}$ is the measured observation vector. $\omega_{oFL},\omega_{oFR},\omega_{oBL},\omega_{oBR}$ is the angular wheel velocity for wheels of the robot in rad/s. $\omega_{gyro}$ is the angular velocity of the robot measured by the gyro in rad/s.

$$
\vec{y} = \begin{bmatrix}
\omega_{oFL} \\
\omega_{oFR} \\
\omega_{oBL} \\
\omega_{oBR} \\
\omega_{\text{gyro}}
\end{bmatrix}
$$

## Bot to Wheel Conversion

We need to make a transformation from the robot reference frame to the wheel reference frame
$r_{FL}, r_{FR}, r_{BL}, r_{BR}$ is the wheel radius in m.

$\theta_{FL}, \theta_{FR}, \theta_{BL}, \theta_{BR}$ is the angle offset from the center in rads.

$l_{FL}, l_{FR}, l_{BL}, l_{BR}$ is the distance to center in m.

$$
    \textbf{T}_{Wheel} = 
    \begin{bmatrix}
        sin(\theta_{FL})/r_{FL} & -cos(\theta_{FL})/r_{FL} & -l_{FL}/r_{FL} \\
        sin(\theta_{FR})/r_{FR} & -cos(\theta_{FR})/r_{FR} & -l_{FR}/r_{FR} \\
        sin(\theta_{BL})/r_{BL} & -cos(\theta_{BL})/r_{BL} & -l_{BL}/r_{BL} \\
        sin(\theta_{BR})/r_{BR} & -cos(\theta_{BR})/r_{BR} & -l_{BR}/r_{BR} \\
    \end{bmatrix}
$$

In [3]:
r = np.full((4,1),0.0245)
theta_deg = np.array([[30, 150, 225, 315]]).T
theta_rads = np.radians(theta_deg)
l = np.full((4,1),0.085)
T_wheel =  -1/r * -np.concatenate((np.sin(theta_rads), np.cos(theta_rads), l), axis = 1) 
print(T_wheel)

[[ 20.40816327  35.34797566   3.46938776]
 [ 20.40816327 -35.34797566   3.46938776]
 [-28.86150127 -28.86150127   3.46938776]
 [-28.86150127  28.86150127   3.46938776]]


## Wheel to Bot Conversion

Pseudo-inverse of $T_{Wheel}$ is the $T_{Bot}$
$$
    \textbf{T}_{Bot} = \textbf{T}^{\dagger}_{Wheel}
$$

In [4]:
T_bot = np.linalg.pinv(T_wheel)
print(T_bot)

[[ 0.01014823  0.01014823 -0.01014823 -0.01014823]
 [ 0.00848705 -0.00848705 -0.00692965  0.00692965]
 [ 0.08442216  0.08442216  0.05969548  0.05969548]]


## Robot Velocity to Wheel Velocity Example

In [5]:
v_example = np.array([1, 1, 0]).T
v_wheel_example = T_wheel*v_example
print(v_wheel_example)

[[ 20.40816327  35.34797566   0.        ]
 [ 20.40816327 -35.34797566   0.        ]
 [-28.86150127 -28.86150127   0.        ]
 [-28.86150127  28.86150127   0.        ]]


# Kalman Filter

In [6]:
process_noise = 0.05 # State noise for unmodeled influences, TODO unclear where is comes from
encoder_sampling_freq = 100 # Hz
encoder_noise_rads = np.radians(0.068) # rads for AS5047P encoder RMS output noise (1 sigma)
encoder_noise = encoder_noise_rads/(1/encoder_sampling_freq) # rads/s TODO unclear if this is valid
gryo_noise_noise_density = 0.014  # deg/s/√Hz for BMI085 Noise density (typ.)
gyro_sampling_freq = 100 # Hz
gyro_noise = np.radians(gryo_noise_noise_density) * np.sqrt(gyro_sampling_freq) # in rad/s

intial_covariance = 0.1 # Picked at random TODO

print(encoder_noise)
print(gyro_noise)

0.11868238913561441
0.002443460952792061


Code below taken from RoboRacing-firmware (written by Peter Wilson)

In [ ]:
dt = 5e-5  #Timestep
end_time = 10 # s

#Define controller parameters (so we can tweak them)
controller_loop_time = 5e-3   #How frequently (in s) the controller runs

#Define data logging arrays
times, dt = np.linspace(0, end_time, num = round(end_time/dt), retstep=True)  #array of times
actual_state_arr = np.zeros(3,1,len(times)) # x_k -> [v_x, v_y, Omega]
estimated_state_arr = np.zeros(len(times)) # \hat{x}
target_state = [1,1,0]
command_input_arr = np.zeros(4,1,len(times)) # x_k -> [omega_ifl, omega_ifr, omega_ibl, omega_ibr]
measured_state_arr = np.zeros(5,1,len(times)) # x_k -> [omega_ofl, omega_ofr, omega_obl, omega_obr, ]

#Now write the simulation code
command_wheel_velocities = [0,0,0,0]
last_controller_execution = 0
last_encoder_speed_calc_time = 0
car_true_vel = start_velocity
encoder_est_speed = 0

for i in range(len(times)):
    time = times[i]
    
    #Find software's current vel
    software_vel_cmd = get_software_vel(time)
    
    #Execute the controller
    if time - last_controller_execution > controller_loop_time:
        timestep = time - last_controller_execution
        
        #estimate speed
        current_reading = get_current_sensor_reading(true_motor_current)
        encoder_est_speed = estimate_vel(timestep, current_reading, command_brakes, num_encoder_ticks_remembered_by_code)
        num_encoder_ticks_remembered_by_code = 0
        
        #Get commands
        command_voltage_brakes = gen_control_voltage_brake_force(timestep, encoder_est_speed, software_vel_cmd)
        command_voltage = command_voltage_brakes.first
        command_brakes = command_voltage_brakes.second
        
        if simulate_brake_update_rate:
            if time - last_brake_cmd_send_time > time_between_brake_updates:
                last_brake_cmd_send_time = time
                sent_brake_command = command_brakes
        else:
            sent_brake_command = command_brakes
        
        last_controller_execution = time
    
    #Bound the control signals
    true_voltage, true_motor_current = get_true_voltage_current(car_true_vel, command_voltage)
    true_brake_force = get_true_braking_force(car_true_vel, sent_brake_command)
    
    #Car physics:
    car_true_position += dt*car_true_vel
    car_true_vel += dt*(-true_gamma_speed*car_true_vel + true_gamma_volt*true_voltage - true_gamma_brake*true_brake_force)
    
    #encoder callback interrupt fires
    if total_num_encoder_ticks/magnets_per_m < car_true_position:
        total_num_encoder_ticks += 1
        num_encoder_ticks_remembered_by_code += 1
    
    #Now do the logs
    vel_kart_arr[i] = car_true_vel
    estimated_vel_arr[i] = get_speed()
    filtered_vel_arr[i] = get_curr_target_speed()
    software_command_velocities_at_timestep_arr[i] = software_vel_cmd
    voltage_cmd_arr[i] = command_voltage
    brake_cmd_arr[i] = command_brakes
    voltage_realized_arr[i] = true_voltage
    brake_realized_arr[i] = true_brake_force
    motor_current_arr[i] = true_motor_current
    error_integral_arr[i] = get_error_integral()